In [2]:
import matplotlib.pyplot as plt
from keras.datasets import imdb
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, Dropout
from keras import callbacks
import keras_tuner

In [4]:
max_features = 10000
maxlen = 500
batch_size = 32
#No reason not to use these values

In [5]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [ ]:
def create_simple_RNN(hp):
    model = Sequential()
    model.add(Embedding(max_features, hp.Int("embedding_neurons", min_value=16, max_value=128, step=16)))
    model.add(SimpleRNN(hp.Int(name="simple_rnn_neurons", min_value=16, max_value=128, step=16), dropout=0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model
create_simple_RNN(keras_tuner.HyperParameters())

In [ ]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=create_simple_RNN,
    objective='val_accuracy')
stop_early = callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(x_train, y_train, validation_split=0.1)

Trial 10 Complete [00h 03m 18s]
val_accuracy: 0.7820000052452087

Best val_accuracy So Far: 0.8131999969482422
Total elapsed time: 00h 40m 13s


In [ ]:
best_hps = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs=20, validation_split=0.1, callbacks=[stop_early])
model.evaluate(x_test, y_test)

Epoch 1/20
704/704 [==============================] - 246s 339ms/step - loss: 0.6900 - accuracy: 0.5304 - val_loss: 0.6707 - val_accuracy: 0.5892
Epoch 2/20
704/704 [==============================] - 235s 333ms/step - loss: 0.6384 - accuracy: 0.6203 - val_loss: 0.8438 - val_accuracy: 0.5512
Epoch 3/20
704/704 [==============================] - 235s 334ms/step - loss: 0.5768 - accuracy: 0.6964 - val_loss: 0.5362 - val_accuracy: 0.7308
Epoch 4/20
704/704 [==============================] - 236s 336ms/step - loss: 0.5099 - accuracy: 0.7493 - val_loss: 0.4563 - val_accuracy: 0.7952
Epoch 5/20
704/704 [==============================] - 237s 336ms/step - loss: 0.4342 - accuracy: 0.8079 - val_loss: 0.4435 - val_accuracy: 0.8004
Epoch 6/20
704/704 [==============================] - 234s 332ms/step - loss: 0.4014 - accuracy: 0.8272 - val_loss: 0.4031 - val_accuracy: 0.8308
Epoch 7/20
704/704 [==============================] - 251s 356ms/step - loss: 0.4205 - accuracy: 0.8194 - val_loss: 0.4031 -

[0.4621240198612213, 0.7858800292015076]

In [9]:
#trying to add LSTM
from keras.layers import LSTM

def create_RNN_with_LSTM(hp):
    model = Sequential()
    model.add(Embedding(max_features, hp.Int("embedding_neurons", min_value=16, max_value=128, step=16)))
    model.add(LSTM(hp.Int(name="simple_rnn_neurons", min_value=16, max_value=128, step=16), dropout=0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model
create_RNN_with_LSTM(keras_tuner.HyperParameters())

In [ ]:
LSTM_tuner = keras_tuner.BayesianOptimization(
    hypermodel=create_RNN_with_LSTM,
    objective='val_accuracy',
    project_name="LSTM")
LSTM_tuner.search(x_train, y_train, validation_split=0.1)

Trial 10 Complete [00h 17m 43s]
val_accuracy: 0.8148000240325928

Best val_accuracy So Far: 0.8628000020980835
Total elapsed time: 04h 14m 40s


In [68]:
best_hps = LSTM_tuner.get_best_hyperparameters()[0]
LSTM_model = LSTM_tuner.hypermodel.build(best_hps)
LSTM_model.fit(x_train, y_train, epochs=20, validation_split=0.1, callbacks=[stop_early])
LSTM_model.evaluate(x_test, y_test)
#A bit better

Epoch 1/20
704/704 [==============================] - 985s 1s/step - loss: 0.4883 - accuracy: 0.7572 - val_loss: 0.4659 - val_accuracy: 0.7904
Epoch 2/20
704/704 [==============================] - 901s 1s/step - loss: 0.3295 - accuracy: 0.8691 - val_loss: 0.3521 - val_accuracy: 0.8564
Epoch 3/20
704/704 [==============================] - 898s 1s/step - loss: 0.2726 - accuracy: 0.8944 - val_loss: 0.3957 - val_accuracy: 0.8572
Epoch 4/20
704/704 [==============================] - 893s 1s/step - loss: 0.2482 - accuracy: 0.9057 - val_loss: 0.2981 - val_accuracy: 0.8792
Epoch 5/20
704/704 [==============================] - 889s 1s/step - loss: 0.2252 - accuracy: 0.9154 - val_loss: 0.3786 - val_accuracy: 0.8508
Epoch 6/20
704/704 [==============================] - 911s 1s/step - loss: 0.2077 - accuracy: 0.9240 - val_loss: 0.3151 - val_accuracy: 0.8884
Epoch 7/20
704/704 [==============================] - 875s 1s/step - loss: 0.1955 - accuracy: 0.9296 - val_loss: 0.3773 - val_accuracy: 0.8816

[0.41203781962394714, 0.8593999743461609]

In [7]:
#trying bidirectional RNN
from keras.layers import Bidirectional

def create_bidirectional_RNN(hp):
    model = Sequential()
    model.add(Embedding(max_features, hp.Int("embedding_neurons", min_value=16, max_value=128, step=16)))
    model.add(Bidirectional(SimpleRNN(hp.Int(name="simple_rnn_neurons", min_value=16, max_value=128, step=16), dropout=0.5)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model
create_bidirectional_RNN(keras_tuner.HyperParameters())

In [70]:
bidirectional_tuner = keras_tuner.BayesianOptimization(
    hypermodel=create_bidirectional_RNN,
    objective='val_accuracy',
    project_name="Bi-RNN")
bidirectional_tuner.search(x_train, y_train, validation_split=0.1)

Trial 10 Complete [00h 05m 16s]
val_accuracy: 0.8180000185966492

Best val_accuracy So Far: 0.8212000131607056
Total elapsed time: 01h 08m 28s


In [71]:
best_hps = bidirectional_tuner.get_best_hyperparameters()[0]
bidirectional_model = bidirectional_tuner.hypermodel.build(best_hps)
bidirectional_model.fit(x_train, y_train, epochs=20, validation_split=0.1, callbacks=[stop_early])
bidirectional_model.evaluate(x_test, y_test)

Epoch 1/20
704/704 [==============================] - 298s 408ms/step - loss: 0.6822 - accuracy: 0.5474 - val_loss: 0.6397 - val_accuracy: 0.6064
Epoch 2/20
704/704 [==============================] - 286s 406ms/step - loss: 0.5102 - accuracy: 0.7537 - val_loss: 0.4534 - val_accuracy: 0.7864
Epoch 3/20
704/704 [==============================] - 291s 414ms/step - loss: 0.4297 - accuracy: 0.8126 - val_loss: 0.3925 - val_accuracy: 0.8396
Epoch 4/20
704/704 [==============================] - 288s 409ms/step - loss: 0.3818 - accuracy: 0.8377 - val_loss: 0.3639 - val_accuracy: 0.8508
Epoch 5/20
704/704 [==============================] - 291s 414ms/step - loss: 0.3713 - accuracy: 0.8468 - val_loss: 0.4570 - val_accuracy: 0.7924
Epoch 6/20
704/704 [==============================] - 288s 409ms/step - loss: 0.3509 - accuracy: 0.8565 - val_loss: 0.3882 - val_accuracy: 0.8488
Epoch 7/20
704/704 [==============================] - 289s 411ms/step - loss: 0.3401 - accuracy: 0.8622 - val_loss: 0.3140 -

[0.3818296790122986, 0.8487600088119507]

In [72]:
#Deep RNN
def create_deep_RNN(hp):
    model = Sequential()
    model.add(Embedding(max_features, hp.Int("embedding_neurons", min_value=16, max_value=128, step=16)))
    for i in range(2):
        model.add(SimpleRNN(hp.Int(name="simple_rnn_neurons", min_value=16, max_value=128, step=16), dropout=0.5, return_sequences=True))
    model.add(SimpleRNN(hp.Int(name="simple_rnn_neurons", min_value=16, max_value=128, step=16)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model
create_deep_RNN(keras_tuner.HyperParameters())

In [77]:
deep_tuner = keras_tuner.BayesianOptimization(
    hypermodel=create_deep_RNN,
    objective='val_accuracy',
    project_name="Deep-RNN")
deep_tuner.search(x_train, y_train, validation_split=0.1)

Trial 10 Complete [00h 10m 42s]
val_accuracy: 0.8248000144958496

Best val_accuracy So Far: 0.8248000144958496
Total elapsed time: 01h 58m 39s


In [78]:
best_hps = deep_tuner.get_best_hyperparameters()[0]
deep_model = deep_tuner.hypermodel.build(best_hps)
deep_model.fit(x_train, y_train, epochs=20, validation_split=0.1, callbacks=[stop_early])
deep_model.evaluate(x_test, y_test)
#More layers don't realy help.

Epoch 1/20
704/704 [==============================] - 646s 894ms/step - loss: 0.6156 - accuracy: 0.6374 - val_loss: 0.5346 - val_accuracy: 0.8024
Epoch 2/20
704/704 [==============================] - 596s 847ms/step - loss: 0.4114 - accuracy: 0.8239 - val_loss: 0.3973 - val_accuracy: 0.8396
Epoch 3/20
704/704 [==============================] - 598s 850ms/step - loss: 0.3618 - accuracy: 0.8501 - val_loss: 0.3626 - val_accuracy: 0.8428
Epoch 4/20
704/704 [==============================] - 578s 821ms/step - loss: 0.3344 - accuracy: 0.8679 - val_loss: 0.4080 - val_accuracy: 0.8568
Epoch 5/20
704/704 [==============================] - 571s 812ms/step - loss: 0.3036 - accuracy: 0.8808 - val_loss: 0.3251 - val_accuracy: 0.8628
Epoch 6/20
704/704 [==============================] - 536s 761ms/step - loss: 0.2859 - accuracy: 0.8881 - val_loss: 0.4407 - val_accuracy: 0.8588
Epoch 7/20
704/704 [==============================] - 529s 751ms/step - loss: 0.2670 - accuracy: 0.8953 - val_loss: 0.4626 -

[0.49983808398246765, 0.7841200232505798]

In [10]:
#Now all together. Except deep.
def create_omega_RNN(hp):
    model = Sequential()
    model.add(Embedding(max_features, hp.Int("embedding_neurons", min_value=16, max_value=128, step=16)))
    model.add(Bidirectional(LSTM(hp.Int(name="simple_rnn_neurons", min_value=16, max_value=128, step=16), dropout=0.5)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model
create_omega_RNN(keras_tuner.HyperParameters())

In [12]:
omega_tuner = keras_tuner.BayesianOptimization(
    hypermodel=create_omega_RNN,
    objective='val_accuracy',
    project_name="Omega_RNN")
omega_tuner.search(x_train, y_train, validation_split=0.1)

Trial 10 Complete [00h 39m 43s]
val_accuracy: 0.819599986076355

Best val_accuracy So Far: 0.847599983215332
Total elapsed time: 04h 35m 31s


In [13]:
stop_early = callbacks.EarlyStopping(monitor='val_loss', patience=5)
best_hps = omega_tuner.get_best_hyperparameters()[0]
omega_model = omega_tuner.hypermodel.build(best_hps)
omega_model.fit(x_train, y_train, epochs=20, validation_split=0.1, callbacks=[stop_early])
omega_model.evaluate(x_test, y_test)

NameError: name 'stop_early' is not defined